In [53]:
def frequency(text: str) -> dict[str, int]:
    text = text.lower()
    freqs: dict[str, int] = {}
    for ch in text:
        if ch in freqs:
            freqs[ch] += 1
        else:
            freqs[ch] = 1
    return freqs

In [54]:
class Node:
    def __init__(self, characters: str, frequency: int):
        self.characters = characters
        self.frequency = frequency
        self.left = None
        self.right = None
    def __lt__(self, other: "Node") -> bool:
        return self.frequency < other.frequency

In [55]:
from heapq import heappush, heappop, heapify

def huffman_tree(freqs: dict[str, int]) -> Node:
    heap: list[Node] = [Node(ch, freq) for ch, freq in freqs.items()]
    heapify(heap)
    if not heap:
        return None
    while len(heap) > 1:
        n1 = heappop(heap) 
        n2 = heappop(heap) 
        parent = Node(n1.characters + n2.characters, n1.frequency + n2.frequency)
        parent.left = n1
        parent.right = n2
        heappush(heap, parent)
    return heap[0]

In [56]:
def get_code(tree: Node, char: str) -> str:
    target = char.lower()
    def search(node: Node, path: str) -> str | None:
        if node is None:
            return None
        if node.left is None and node.right is None:
            if node.characters == target:
                return path
            else:
                return None
        if node.left and target in node.left.characters:
            res = search(node.left, path + "0")
            if res is not None:
                return res
        if node.right and target in node.right.characters:
            res = search(node.right, path + "1")
            if res is not None:
                return res
        return None
    code = search(tree, "")
    if code is None:
        raise ValueError(f"Character {char!r} not found in tree")
    return code

In [57]:
def show_all_codes(tree: Node) -> None:
    if tree is None:
        return
    characters = set(tree.characters)
    for char in sorted(characters):
        code = get_code(tree, char)
    print(f"Character: {char} Code: {code}")

In [76]:
def huffman_encode(text: str, tree: Node) -> str:
    bits: list[str] = []
    for ch in text.lower():
        bits.append(get_code(tree, ch))
    return "".join(bits)

In [77]:
def huffman_decode(encoded_text: str, tree: Node) -> str:
    result_chars: list[str] = []
    node = tree
    for bit in encoded_text:
        if bit == "0":
            node = node.left
        elif bit == "1":
            node = node.right
        else:
            raise ValueError
        if node.left is None and node.right is None:
            result_chars.append(node.characters)
            node = tree 
    return "".join(result_chars)

In [78]:
freqs_english = {
" ": 18.0,
"e": 12.02, "t": 9.10, "a": 8.12, "o": 7.68, "i": 7.31, "n": 6.95,
"s": 6.28, "r": 6.02, "h": 5.92, "d": 4.32, "l": 3.98, "u": 2.88,
"c": 2.71, "m": 2.61, "f": 2.30, "y": 2.11, "w": 2.09, "g": 2.03,
"p": 1.82, "b": 1.49, "v": 1.11, "k": 0.69, "x": 0.17, "q": 0.11,
"j": 0.10, "z": 0.07
}
english_tree = huffman_tree(freqs_english)
test_text = "huffman coding is a data compression algorithm"
encoded = huffman_encode(test_text, english_tree)
decoded = huffman_decode(encoded, english_tree)
print("Test text:", test_text)
print("Encoded length:", len(encoded), "bits")
print("Original length:", len(test_text) * 8, "bits")
print("Compression ratio:", round((1 - len(encoded)/(len(test_text)*8))*100, 1), "%")
print("Decoded matches original:", decoded == test_text.lower())

Test text: huffman coding is a data compression algorithm
Encoded length: 199 bits
Original length: 368 bits
Compression ratio: 45.9 %
Decoded matches original: True
